<a href="https://colab.research.google.com/github/ymuto0302/ML/blob/main/MachineLearning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sklearn を用いた分類実験：最初の一歩
sklearn (scikit-learn) を用いると，データセットの読み込み／モデルの構築／評価が簡単に行える。

目的：
- iris データセットを対象として「分類実験の流れ」を把握する。
- 分類手法を他のデータセットへ適用し，サンプル数が少ない場合や高次元空間での分類を試みる。

---
## iris データセット
iris データセットは３種のアヤメ (iris) を分類する問題である。下記のサイトにおいて，iris データセットの特性が説明されている。

https://machinelearninghd.com/iris-dataset-uci-machine-learning-repository-project/

- サンプル数：150個 (50個／クラス)
- 次元数：4次元
- クラス：0〜2 の 3クラス

sklearn.datasets.load_iris() のマニュアル：  
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris

最初に，データセットを読み込む。

In [1]:
# iris データセットの読み込み
from sklearn.datasets import load_iris
iris = load_iris()

データセットの中身を見てみよう。

In [3]:
# iris データセットの説明
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

データおよびターゲットの先頭５個を見てみると，データは４次元，ターゲットには数値が入っていることが分かる。

In [36]:
# データの先頭
iris.data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [40]:
# データの各次元の意味
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [37]:
# 各サンプルのクラス番号
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [38]:
# クラス番号の意味
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [4]:
# その他の諸元
print("データの data type:{}".format(type(iris.data)))
print("ターゲットの data type:{}".format(type(iris.target)))
print("データの次元数:{}".format(iris.data.shape))
print("ターゲットの次元数:{}".format(iris.target.shape))

データの data type:<class 'numpy.ndarray'>
ターゲットの data type:<class 'numpy.ndarray'>
データの次元数:(150, 4)
ターゲットの次元数:(150,)


(参考) データ，ターゲットともに numpy 配列である。

In [5]:
# (参考) サンプルのデータとクラス番号を並べて表示してみる
for data , target in zip(iris.data[:5], iris.target[:5]):
    print(data, target)

[5.1 3.5 1.4 0.2] 0
[4.9 3.  1.4 0.2] 0
[4.7 3.2 1.3 0.2] 0
[4.6 3.1 1.5 0.2] 0
[5.  3.6 1.4 0.2] 0


---
## 分類実験

### データを学習用とテスト用に分割
各クラスのバランスを取りながら分割するには `sklearn.model_section.train_test_split()` を用いる。

`sklearn.model_section.train_test_split` のマニュアル：  
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [12]:
from sklearn.model_selection import train_test_split

# 学習:テスト=7:3 の比率に分割
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

- `X_train`, `X_test` に４次元データが入る。
- `y_train`, `y_test` に１次元のラベルが入る。

`X_train`, `X_test` のサイズ(shape)を確認すると，学習用に 105サンプル，テスト用に 45サンプルが振り分けられていることが分かる。  
また，`y_train`, `y_test` を確認すると，クラス 0, 1, 2 が等分に割り当てられている。

In [14]:
print(X_train.shape, X_test.shape)

(105, 4) (45, 4)


In [17]:
print(y_train)

[2 2 2 1 1 1 2 1 2 0 0 0 0 2 1 0 2 1 1 1 1 2 2 0 2 2 0 1 1 2 1 2 2 1 2 2 1
 2 0 1 2 0 0 2 1 0 2 1 0 0 0 0 0 1 2 2 2 2 0 2 0 1 2 1 2 0 0 0 2 2 2 1 1 0
 0 2 0 1 1 0 2 2 0 1 2 0 1 2 2 0 1 0 2 0 0 0 0 2 1 1 0 2 1 1 1]


In [18]:
print(y_test)

[2 0 1 1 2 1 2 1 1 2 0 0 1 0 1 1 1 2 0 1 0 0 0 2 0 0 2 0 2 0 0 1 1 1 2 0 2
 1 2 0 1 1 1 0 1]


### モデルの学習と分類実験
分類器として SVC (Support Vector Classification; SVM (Support Vector Machine) の分類バージョン) を利用する。

(参考) SVM は回帰にも利用可能であり，この時，SVR (Support Vector Regression) と呼ばれる。

sklearn におけるモデルの学習とテストは極めて簡単である。
1. `モデル.fit()` ・・・モデルの学習
1. `モデル.prediction()` ・・・モデルによる分類
1. 正解率などのメトリックを用いたモデルの評価


In [41]:
from sklearn.svm import SVC

# モデルの学習と評価
model = SVC(kernel='rbf') # モデルの定義
model.fit(X_train, y_train) # モデルの学習
pred = model.predict(X_test) # モデルの評価（分類）
print(pred)

[2 0 2 1 2 1 2 1 1 2 0 0 1 0 1 1 1 2 0 1 0 0 0 2 0 0 2 0 2 0 0 1 1 1 2 0 2
 1 2 0 1 1 1 0 1]


分類結果を評価するには `sklearn.metrics` を用いる。

In [25]:
# 正解率
from sklearn.metrics import accuracy_score
pred = model.predict(X_test)
print("accuracy: {}".format(accuracy_score(y_test, pred)))

# confusion matrix
from sklearn.metrics import confusion_matrix
print("\n confusion matrix")
print(confusion_matrix(y_test, pred))

accuracy: 0.9777777777777777

 confusion matrix
[[16  0  0]
 [ 0 17  1]
 [ 0  0 11]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.94      0.97        18
           2       0.92      1.00      0.96        11

    accuracy                           0.98        45
   macro avg       0.97      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



In [44]:
'''
データセットの分割／モデル定義・学習・分類／分類結果の分析をまとめたコード
'''
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# iris データセットの読み込み
from sklearn.datasets import load_iris
iris = load_iris()

# 学習:テスト=7:3 の比率に分割
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

# モデルの学習と評価
model = SVC(kernel='rbf') # モデルの定義
model.fit(X_train, y_train) # モデルの学習
pred = model.predict(X_test) # モデルの評価（分類）

# 正解率
print("accuracy: {}".format(accuracy_score(y_test, pred)))

# confusion matrix
print("\n confusion matrix")
print(confusion_matrix(y_test, pred))

accuracy: 0.9111111111111111

 confusion matrix
[[10  0  0]
 [ 0 20  3]
 [ 0  1 11]]


---
# 他のデータセットを用いた分類実験
次の wine dataset / digits dataset を対象に分類実験をやってみよう。

## wine dataset
sklearn.datasets.load_wine()
- サンプル数：178個
- 次元数：13次元
- クラス：0〜2 の 3クラス

## digits dataset
sklearn.datasets.load_digits()
- サンプル数：1797個
- 次元数：64次元
- クラス：0〜9 の 10クラス

(参考)
- sklearn.datasets.load_wine() のマニュアル：   
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine
- sklearn.datasets.load_digits() のマニュアル：   
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits

